In [1]:
!pip install -qU langchain-community wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmanager 0.7.1 requires sqlalchemy==1.2.19, but you have sqlalchemy 2.0.46 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.2.7 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.1.0 which is incompatible.


In [2]:
import re
from collections import Counter
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()

In [3]:
def ExtractContentFromWikipedia(query="Nigerian History"):
    txt = ""
    for docs in retriever.invoke("Nigerian History"):
        page_content = docs.page_content
        txt+= " " + page_content
    return txt.lower()

extracted_txt = ExtractContentFromWikipedia()

In [4]:
init_word_list = re.findall(pattern=r"\w+|[^\s\w]+", string=extracted_txt, flags=re.MULTILINE)

init_word_list[:5]

['the', 'history', 'of', 'nigeria', 'can']

In [5]:
word_count = Counter(init_word_list) # Returns a dictionary of word and its counts

word_count.most_common(10)

[(',', 142),
 ('the', 126),
 ('.', 114),
 ('of', 83),
 ('in', 68),
 ('and', 62),
 ('-', 47),
 ('nigeria', 32),
 ('to', 29),
 ('west', 29)]

In [6]:
eos_token = "</end>"

current_wrd_split = {}
for word in word_count:
    wrd_split = list(word)+[eos_token]
    current_wrd_split[word] = wrd_split

#### OUTPUT #### 
# {'beginning': ['b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g', '</w>']}

In [7]:
unique_token = set()
for word in current_wrd_split:
    wrd_lst = current_wrd_split[word]
    unique_token.update(wrd_lst)

#### OUTPUT ####
# Initial vocabulary created with 56 unique symbols.
# Initial vocabulary symbols: ['"','&',"'",'(',')','+', ',','-','.','/',
# '0','1','2','3','4','5','6','7','8','9',':',';','</end>','=','a','b',
# 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t',
# 'u','v','w','x','y','z','é','ü','̀','ẹ','–','—']

In [ ]:
def TrainTokenizer(num_merges=75):
    num_epochs = num_merges 
    learned_merges = {}

    pair_count = Counter()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        for token, freq in word_count.items():
            wrd_split = current_wrd_split[token]
            for idx in range(1, len(wrd_split)):
                char_pair = (wrd_split[idx-1], wrd_split[idx])
                pair_count[char_pair] += freq
        if not pair_count:
            print("No more pairs found to merge. Stopping early.")
            break

        best_pair = max(pair_count, key=pair_count.get)
        print(f"Found best pair: {best_pair} with frequency {pair_count[best_pair]}")

        learned_merges[best_pair] = epoch

        new_token = "".join(best_pair)

        print(new_token)
        break

In [ ]:
TrainTokenizer()